In [1]:
import numpy as np
import pandas as pd
import json
import io
import os
import sys
import pickle
import math
import random
from time import time

import matplotlib.pyplot as plt

In [8]:
NO_CLIENTS = 210
NO_GROUPS = 7
NO_CLASSES = 10
ALPHA = "03"

replace_dict = {
    0: 7,
    1: 6,
    2: 5,
    3: 4,
    4: 3,
    5: 2,
    6: 1,
}

path = f"../data/alpha/Cifar10_NIID_{NO_CLIENTS}c_a{ALPHA}/config.json"
conf = json.loads(open(path, "r").read())
data = [dict(zip(np.array(cli)[:,0], np.array(cli)[:,1])) for cli in conf["Size of samples for labels in clients"]]

main_label_vectors = np.zeros((NO_CLIENTS,NO_CLASSES))
for client_id in range(NO_CLIENTS):
    for class_id in range(NO_CLASSES):
        if class_id in data[client_id].keys():
            main_label_vectors[client_id][class_id] = data[client_id][class_id]

def spread_bw_random(main_label_vectors, no_clients, tiers):
    NO_CLIENTS = no_clients
    
    bw_types = []
    bw = []
    for k, v in tiers.items():
        for i in range(int(NO_CLIENTS / len(tiers))):
            bw_types.append(k)
            bw.append(random.choice(tiers[k]))

    # lows = [weak] * low_count
    # mediums = [normal] * medium_count
    # highs = [strong] * high_count
    # bw = lows + mediums + highs

    print(int(NO_CLIENTS / len(tiers)))
    print(len(bw))
    print(len(bw_types))

    ids = np.arange(NO_CLIENTS)
    random.shuffle(ids)

    bws = []
    for idx in ids:
        bws.append((bw[idx],bw_types[idx]))

    clients_list = []
    for idx in range(len(main_label_vectors)):
        clients_list.append(
            {
                "client_id" : idx,
                "bw" : bws[idx][0],
                "bw_type" : bws[idx][1],
                # "distribution" : main_label_vectors[idx],
                # "group" : None
            }
        )

    clients_df = pd.DataFrame(clients_list)

    maindf = clients_df.sort_values("bw")
    # maindf["group"] = np.repeat(np.arange(7),int(NO_CLIENTS / NO_GROUPS))


    # for idx,row in maindf.iterrows():
    #     maindf.at[idx,"group"] = replace_dict[row.group]
    #     # maindf.at[idx,"distribution"] = np.array([float(item) for item in row.distribution.replace("[","").replace("]","").split(" ") if item != ""])
        
    maindf.reset_index(drop=True)

    return maindf

In [64]:
tiers = {
    "TIER_11" : np.arange(2_500_000, 3_500_000, 500_000),
    "TIER_12" : np.arange(4_000_000, 5_500_000, 500_000),
    "TIER_13" : np.arange(6_000_000, 8_000_000, 500_000),

    "TIER_21" : np.arange(8_500_000, 12_500_000, 1_000_000),
    "TIER_22" : np.arange(13_000_000, 16_000_000, 1_000_000),
    "TIER_23" : np.arange(17_000_000, 25_000_000, 1_000_000),
    "TIER_24" : np.arange(26_000_000, 30_000_000, 1_000_000),
    "TIER_25" : np.arange(31_000_000, 34_000_000, 1_000_000),

    "TIER_31" : np.arange(35_000_000, 40_000_000, 1_000_000),
    "TIER_32" : np.arange(41_000_000, 55_000_000, 1_000_000),
    "TIER_33" : np.arange(46_000_000, 65_000_000, 1_000_000),
    "TIER_34" : np.arange(66_000_000, 75_000_000, 1_000_000),
    "TIER_35" : np.arange(75_000_000, 100_000_000, 1_000_000),

    "TIER_41" : np.arange(100_000_000, 400_000_000, 20_000_000)
}

main_df = spread_bw_random(main_label_vectors, 210, tiers)

assignment = {
    "block_0": {},
    "block_1": {},
    "block_2": {},
    "block_3": {},
    "block_4": {},
    "block_5": {},
    "block_6": {},
}

for bid in assignment.keys():
    selected_clients = random.sample(list(main_df.client_id.values), 30)
    assignment[bid] = main_df.iloc[selected_clients]["bw_type"].to_dict()
    assignment[bid] = {str(k):v for k,v in assignment[bid].items()} 

with open(f"../data/alpha/fedavg/Cifar10_NIID_210c_a03/client_groups.json","w") as f:
    json.dump(assignment,f)

15
210
210


{'block_0': {'174': 'TIER_35',
  '107': 'TIER_32',
  '183': 'TIER_33',
  '63': 'TIER_25',
  '118': 'TIER_31',
  '80': 'TIER_23',
  '79': 'TIER_23',
  '82': 'TIER_13',
  '137': 'TIER_23',
  '161': 'TIER_25',
  '41': 'TIER_34',
  '208': 'TIER_34',
  '120': 'TIER_22',
  '0': 'TIER_21',
  '144': 'TIER_22',
  '112': 'TIER_12',
  '155': 'TIER_34',
  '121': 'TIER_22',
  '192': 'TIER_25',
  '164': 'TIER_22',
  '184': 'TIER_32',
  '201': 'TIER_13',
  '147': 'TIER_12',
  '74': 'TIER_35',
  '2': 'TIER_24',
  '108': 'TIER_11',
  '197': 'TIER_11',
  '191': 'TIER_32',
  '124': 'TIER_25',
  '204': 'TIER_41'},
 'block_1': {'163': 'TIER_22',
  '88': 'TIER_13',
  '107': 'TIER_32',
  '38': 'TIER_13',
  '99': 'TIER_32',
  '152': 'TIER_21',
  '145': 'TIER_25',
  '199': 'TIER_11',
  '95': 'TIER_31',
  '12': 'TIER_13',
  '58': 'TIER_23',
  '202': 'TIER_25',
  '16': 'TIER_34',
  '169': 'TIER_41',
  '112': 'TIER_12',
  '43': 'TIER_25',
  '148': 'TIER_25',
  '47': 'TIER_41',
  '144': 'TIER_22',
  '63': 'TIER_25

In [69]:
sorted(main_df.client_id.values)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
